# 1. `groupby`를 통한 Categorical 데이터의 통계치 계산
- Pandas의 `groupby`는 데이터를 특정 카테고리로 그룹화한 후 각 그룹에 대해 집계(aggregation), 변환(transformation), 필터링(filtering) 등의 연산을 수행할 수 있게 해줍니다.
- 이 과정은 일반적으로 세 단계로 이루어집니다.

| groupby 개념도 | 설명 |
|----------|----------|
| <img src="../images/groupby_image.png" width="400" height="250"></img>    |  1. **Splitting (분할)**: 데이터를 특정 키(key) 기준으로 분할합니다. <br> 2. **Applying (적용)**: 각 그룹에 함수를 적용하여 결과를 얻습니다. <br>3. **Combining (결합)**: 함수의 결과를 하나의 데이터 구조로 결합합니다. |

- `groupby` 연산은 트레이딩에 필요한 대용량 데이터를 다룰 때 매우 유용합니다.



## 1) GroupBy 사용 방법
`groupby` 메소드는 DataFrame 객체에서 사용할 수 있습니다. 기본적인 사용 방법은 다음과 같습니다.

In [1]:
import pandas as pd

In [2]:
path = "../dataset/titanic.csv"
df = pd.read_csv(path, index_col=0)
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,67,1,2,"Nye, Mrs. (Elizabeth Ramell)",female,29.0,0,0,C.A. 29395,10.5000,S
1,357,1,1,"Bowerman, Miss. Elsie Edith",female,22.0,0,1,113505,55.0000,S
2,287,1,3,"de Mulder, Mr. Theodore",male,30.0,0,0,345774,9.5000,S
3,859,1,3,"Baclini, Mrs. Solomon (Latifa Qurban)",female,24.0,0,3,2666,19.2583,C
4,356,0,3,"Vanden Steen, Mr. Leo Peter",male,28.0,0,0,345783,9.5000,S


In [3]:
# 좌석등급 기준으로 살아남은 사람 카운트
df.groupby("Pclass")["Survived"].sum().reset_index()

,Pclass,Survived
0,1,56
1,2,37
2,3,39


In [4]:
# 생존 여부에 따른 티켓 가격 평균구하기
df.groupby("Survived")["Fare"].mean().reset_index()

,Survived,Fare
0,0,25.138143
1,1,55.243720


In [5]:
# 좌석등급, 생존 여부에 따른 티켓 가격 평균구하기
df.groupby(["Pclass", "Survived"])["Fare"].mean().reset_index()

,Pclass,Survived,Fare
0,1,0,87.190574
1,1,1,105.595166
2,2,0,19.016667
3,2,1,23.522524
4,3,0,14.324563
5,3,1,13.038674


- groupby가 split, apply, combine을 통해 계산되는 과정

In [6]:
# groupby 연산이 특정 열(Pclass)의 값 종류별(1,2,3)로 dataframe을 split함
grouped = df.groupby("Pclass")
for k, item in grouped:
    print(k, item.shape)
    display(item.head(4))
    print("="*50)

1 (79, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
1,357,1,1,"Bowerman, Miss. Elsie Edith",female,22.0,0,1,113505,55.0000,S
5,797,1,1,"Leader, Dr. Alice (Farnham)",female,49.0,0,0,17465,25.9292,S
14,631,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0,0,0,27042,30.0000,S
15,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0000,S


2 (67, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,67,1,2,"Nye, Mrs. (Elizabeth Ramell)",female,29.0,0,0,C.A. 29395,10.50,S
7,583,0,2,"Downton, Mr. William James",male,54.0,0,0,28403,26.00,S
9,627,0,2,"Kirkland, Rev. Charles Leonard",male,57.0,0,0,219533,12.35,Q
10,751,1,2,"Wells, Miss. Joan",female,4.0,1,1,29103,23.00,S


3 (154, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
2,287,1,3,"de Mulder, Mr. Theodore",male,30.0,0,0,345774,9.5000,S
3,859,1,3,"Baclini, Mrs. Solomon (Latifa Qurban)",female,24.0,0,3,2666,19.2583,C
4,356,0,3,"Vanden Steen, Mr. Leo Peter",male,28.0,0,0,345783,9.5000,S
6,510,1,3,"Lang, Mr. Fang",male,26.0,0,0,1601,56.4958,S


In [7]:
# 특정 열(Pclass)의 값 종류(1,2,3)로 쪼개진 dataframe을
# 보고자 하는 열(Survived)에 대해서 값을 연산(sum) 후(apply)
# combine을 통해 하나의 dataframe으로 합침(combine)
grouped["Survived"].sum()

Pclass
1    56
2    37
3    39
Name: Survived, dtype: int64

## 2) 집계 함수
그룹화된 데이터에 적용할 수 있는 집계 함수들은 다음과 같습니다.

- `sum()`: 합계
- `mean()`: 평균
- `median()`: 중앙값
- `min()`: 최소값
- `max()`: 최대값
- `count()`: 개수
- `size()`: 그룹의 크기
- `std()`: 표준편차
- `var()`: 분산
- `first()`: 그룹의 첫 번째 값
- `last()`: 그룹의 마지막 값
- `nth()`: n번째 값
- `agg()`: 여러 집계 함수를 동시에 적용

원하는 연산은 `sum()`과 같이 직접 호출하거나, `agg()` 함수 안에 인자로 사용할 수 있습니다.

In [8]:
grouped = df.groupby("Pclass")

In [9]:
# Pclass 별로 나이의 평균 계산
grouped['Age'].mean().reset_index()

,Pclass,Age
0,1,37.859747
1,2,29.258657
2,3,23.974026


In [10]:
# agg() 함수를 사용해서 Pclass 별 Age의 min, max 구하기
grouped["Age"].agg(["min", "max"])

,min,max
Pclass,,
1,0.92,80.0
2,0.83,70.0
3,1.00,70.5


In [11]:
# agg() 함수를 사용하여 원하는 열 마다 다른 집계함수 적용
# e.g. Pclass별로 Fare열의 평균 및 표준편차, Age열의 min, max 구하기
grouped.agg({
    'Fare': ['mean', 'std'],
    'Age': ['min', 'max']
})

Fare              Age      
              mean        std   min   max
Pclass                                   
1       100.236867  81.237675  0.92  80.0
2        21.504976  12.974632  0.83  70.0
3        13.998916  10.367883  1.00  70.5

## 3) transform()과 filter() 사용하기
`transform()`: 
- `groupby` 연산 결과를 원래 datafraem과 동일한 크기로 반환합니다. 

In [12]:
grouped = df.groupby("Pclass")

In [13]:
grouped['Fare'].mean()

Pclass
1    100.236867
2     21.504976
3     13.998916
Name: Fare, dtype: float64

In [14]:
grouped['Fare'].transform('mean')

0       21.504976
1      100.236867
2       13.998916
3       13.998916
4       13.998916
          ...    
295    100.236867
296    100.236867
297     21.504976
298     13.998916
299    100.236867
Name: Fare, Length: 300, dtype: float64

In [15]:
# Pclass 그룹별로 평균을 구한 다음
# 원래 데이터의 row별 Pclass별 Fare의 평균값을 지닌 열을 기존의 dataframe에 추가 
df["Average Fare by Pclass"] = grouped['Fare'].transform('mean')

In [16]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Average Fare by Pclass
0,67,1,2,"Nye, Mrs. (Elizabeth Ramell)",female,29.0,0,0,C.A. 29395,10.5000,S,21.504976
1,357,1,1,"Bowerman, Miss. Elsie Edith",female,22.0,0,1,113505,55.0000,S,100.236867
2,287,1,3,"de Mulder, Mr. Theodore",male,30.0,0,0,345774,9.5000,S,13.998916
3,859,1,3,"Baclini, Mrs. Solomon (Latifa Qurban)",female,24.0,0,3,2666,19.2583,C,13.998916
4,356,0,3,"Vanden Steen, Mr. Leo Peter",male,28.0,0,0,345783,9.5000,S,13.998916


### (2) `filter()`:
- `filter` 메소드는 함수를 각 그룹에 적용하여, 해당 함수의 반환 값이 True인 그룹만을 필터링합니다

In [17]:
grouped = df.groupby("Pclass")

In [18]:
# Pclass별 승객 수가 80명 이상인 클래스만을 필터링하기
filtered = grouped.filter(lambda x: len(x) >= 80)
filtered.shape

(154, 12)

In [19]:
# Pclass별 승객 수 확인해보기
grouped = df.groupby("Pclass")
for k, item in grouped:
    print(k, item.shape)


1 (79, 12)
2 (67, 12)
3 (154, 12)
